In [ ]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


In [ ]:
df = pd.read_csv("IMDB_Dataset.csv")  # Ensure the file is in your working directory
print(df.head())
print(df['sentiment'].value_counts())


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
    return text

df['review'] = df['review'].apply(clean_text)

In [ ]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['sentiment'], test_size=0.2, random_state=42
)

In [ ]:
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1,2), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()


print("TF-IDF shape:", X_train_tfidf.shape)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.regularizers import l2  # 👈 Add this line

model = Sequential([
    Dense(1024, kernel_regularizer=l2(0.001), input_shape=(X_train_tfidf.shape[1],)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.5),

    Dense(512, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.4),

    Dense(256, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.3),

    Dense(128, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])


In [ ]:
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping  

early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=2, 
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    X_train_tfidf, y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
y_pred = (model.predict(X_test_tfidf) > 0.5).astype("int32")

print("\n✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n🔍 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
# ============================================
# 💾 Save Model and TF-IDF Vectorizer
# ============================================
import joblib

# Save the trained model
model.save("IMDB_Sentiment_Classifier.h5")
print("✅ Model saved successfully as 'IMDB_Sentiment_Classifier.h5'")

# Save the TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("✅ TF-IDF Vectorizer saved successfully as 'tfidf_vectorizer.pkl'")


In [ ]:
from tensorflow.keras.models import load_model
import joblib

# Load model and vectorizer
model = load_model("IMDB_Sentiment_Classifier.h5")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Example prediction
sample_text = ["This movie was absolutely fantastic!"]
sample_tfidf = vectorizer.transform(sample_text).toarray()
prediction = (model.predict(sample_tfidf) > 0.5).astype("int32")

print("🧠 Predicted Sentiment:", "Positive" if prediction[0][0] == 1 else "Negative")
